# Confidence Intervals
<hr>
A confidence interval is a range is which we are confident to a certain degree that a specific population paramter will fall.

In this notebook every $\sigma$ represents a population standard deviation while $s$ represents a sample standard deviation.